<a href="https://colab.research.google.com/github/jakobbaumgartner/CNN-HotDog-Classifier/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
import tensorflow as tf

In [111]:
tf.keras.__version__

'2.4.0'

In [112]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

In [113]:
use_tpu = True #@param {type:"boolean"}

if use_tpu:
    assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  TF_MASTER=''

In [114]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TF_MASTER)
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.3.215.250:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.3.215.250:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Nastavimo poti do fotografij, uredimo fotografije. In izdelamo training dataset.

In [115]:
categories = ["hot_dog", "not_hot_dog"]
datadir = "drive/MyDrive/Colab Notebooks/seefood/train"
os.getcwd()
IMG_SIZE = 200 # maybe change this setting later, if the results are not good

training_data = []

for category in categories:  # do hot_dog and not_hot_dog

    path = os.path.join(datadir,category)  # create path to hotdogs
    class_num = categories.index(category)  # get the classification  (0 or a 1). hot or not

    for img in tqdm(os.listdir(path)):  # iterate over each image 
        try:
            img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to grayscale array
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) # resize 
            training_data.append([new_array, class_num]) 
          
        except Exception as e:  # in the interest in keeping the output clean...
              print(e)
      
    

100%|██████████| 449/449 [00:02<00:00, 205.72it/s]


Sedaj želimo premešati podatke, da niso vse slike kategorij po vrsti, drugače imamo težave z učenjem.

In [116]:
import random
random.shuffle(training_data)

In [117]:
for sample in training_data[:10]: # lets check if it is well shuffled 
    print(sample[1])

1
1
1
0
1
1
1
0
1
1


In [118]:
X = []   # features (big X)
y = []   # labels (small y)

for features,label in training_data:
    X.append(features) 
    y.append(label)   # labels

print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

# we have to make features into a np array
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1) # number of features, size_x, size_y, colors (grayscale = 1)

[[[[ 24]
   [ 25]
   [ 25]
   ...
   [137]
   [142]
   [145]]

  [[ 26]
   [ 26]
   [ 27]
   ...
   [138]
   [144]
   [145]]

  [[ 24]
   [ 26]
   [ 27]
   ...
   [140]
   [146]
   [147]]

  ...

  [[ 63]
   [ 62]
   [ 67]
   ...
   [113]
   [118]
   [116]]

  [[ 57]
   [ 60]
   [ 63]
   ...
   [110]
   [112]
   [106]]

  [[ 57]
   [ 60]
   [ 61]
   ...
   [105]
   [114]
   [110]]]]


 *The basic CNN structure is as follows: Convolution -> Pooling -> Convolution -> Pooling -> Fully Connected Layer -> Output*

*Convolution is the act of taking the original data, and creating feature maps from it.Pooling is down-sampling, most often in the form of "max-pooling," where we select a region, and then take the maximum value in that region, and that becomes the new value for the entire region. Fully Connected Layers are typical neural networks, where all nodes are "fully connected." The convolutional layers are not fully connected like a traditional neural network.* 

Na tej točki začnemo zares načrtovati nevronsko mrežo.

In [119]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [120]:
X=np.array(X/255.0)
y=np.array(y)



In [ ]:
model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=64, epochs=16, validation_split=0.3)

Epoch 1/16
10/10 [==============================] - 32s 3s/step - loss: 1.9558 - accuracy: 0.4294 - val_loss: 0.7158 - val_accuracy: 0.4741
Epoch 2/16
10/10 [==============================] - 30s 3s/step - loss: 0.7040 - accuracy: 0.4756 - val_loss: 0.6935 - val_accuracy: 0.4741
Epoch 3/16
 1/10 [==>...........................] - ETA: 25s - loss: 0.6941 - accuracy: 0.3906